# 프로젝트 - Movielens 영화 추천 실습

목표 - MF 모델 학습 방법을 토대로, 내가 좋아할 만한 영화 추천 시스템을 제작해 보기

- 유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있다. MovieLens 1M Dataset 사용을 권장.
- 별점 데이터는 대표적인 explicit 데이터. 하지만 implicit 데이터로 간주하고 테스트.
- 별점을 시청횟수로 해석.
- 또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외.

## 1) 데이터 준비와 전처리

In [45]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [46]:
orginal_data_size

1000209

### 3점 이상만 추출

In [47]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [48]:
# rating 컬럼의 이름을 counts로 바꿉니다.
ratings.rename(columns={'rating':'counts'}, inplace=True)

In [49]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

### 사용할 컬럼만 추출하여 저장

In [50]:
ratings = ratings[['user_id','movie_id','counts']]

In [51]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [52]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  3883 non-null   int64 
 1   title     3883 non-null   object
 2   genre     3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


### ratings 와 movies 데이터 프레임 조인

In [81]:
ratings_movies = ratings.merge(movies, on='movie_id', how='left')

In [82]:
len(ratings_movies) #ratings 데이터프레임과 같은 숫자로 left join 된 것을 확인

836478

In [136]:
ratings_movies

,user_id,movie_id,counts,title,genre
0,1,1193,5,one flew over the cuckoo's nest (1975),Drama
1,1,661,3,james and the giant peach (1996),Animation|Children's|Musical
2,1,914,3,my fair lady (1964),Musical|Romance
3,1,3408,4,erin brockovich (2000),Drama
4,1,2355,5,"bug's life, a (1998)",Animation|Children's|Comedy
...,...,...,...,...,...
836473,6040,1090,3,platoon (1986),Drama|War
836474,6040,1094,5,"crying game, the (1992)",Drama|Romance|War
836475,6040,562,5,welcome to the dollhouse (1995),Comedy|Drama
836476,6040,1096,4,sophie's choice (1982),Drama


In [146]:
traing_data = ratings_movies[['user_id','title','counts']] #훈련시 사용하게 될 데이터 컬럼을 추출해 놓음.

## 2) 분석해 봅시다.

### ratings에 있는 유니크한 영화 개수

In [83]:
num_movie = ratings_movies['movie_id'].nunique()
num_movie

3628

### rating에 있는 유니크한 사용자 수

In [84]:
num_user = ratings_movies['user_id'].nunique()
num_user

6039

### 가장 인기 있는 영화 30개(인기순)

In [85]:
movie_counts = ratings_movies.groupby('movie_id')['counts'].count()
movie_counts_top30 = movie_counts.sort_values(ascending=False).head(30)

In [86]:
movie_counts_top30.head(5)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
Name: counts, dtype: int64

In [87]:
df = pd.DataFrame(data = movie_counts_top30
                ,index = movie_counts_top30.index)

인기 있는 영화 30개가 어떤 영화인지 이름 검색을 데이터프레임 조인으로 확인

In [88]:
movie_counts_top30 = df.merge(movies, on='movie_id', how='left')
movie_counts_top30

,movie_id,counts,title,genre
0,2858,3211,American Beauty (1999),Comedy|Drama
1,260,2910,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
2,1196,2885,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
3,1210,2716,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
4,2028,2561,Saving Private Ryan (1998),Action|Drama|War
5,589,2509,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller
6,593,2498,"Silence of the Lambs, The (1991)",Drama|Thriller
7,1198,2473,Raiders of the Lost Ark (1981),Action|Adventure
8,1270,2460,Back to the Future (1985),Comedy|Sci-Fi
9,2571,2434,"Matrix, The (1999)",Action|Sci-Fi|Thriller


## 3) 내가 선호하는 영화를 5가지 골라서 rating에 추가해 줍시다.

In [147]:
traing_data.shape

(836478, 3)

In [148]:
traing_data['title'] = traing_data['title'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
traing_data.head(10)

/home/ssac6/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,user_id,title,counts
0,1,one flew over the cuckoo's nest (1975),5
1,1,james and the giant peach (1996),3
2,1,my fair lady (1964),3
3,1,erin brockovich (2000),4
4,1,"bug's life, a (1998)",5
5,1,"princess bride, the (1987)",3
6,1,ben-hur (1959),5
7,1,"christmas story, a (1983)",5
8,1,snow white and the seven dwarfs (1937),4
9,1,"wizard of oz, the (1939)",4


In [149]:
traing_data.info() #정보를 추가히기 전에 컬럼 타입이 무엇인지 확인

<class 'pandas.core.frame.DataFrame'>
Int64Index: 836478 entries, 0 to 836477
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   user_id  836478 non-null  int64 
 1   title    836478 non-null  object
 2   counts   836478 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 65.5+ MB


In [120]:
# 내가 좋아 하는 영화
favorite_movie = ratings_movies[ratings_movies['title'].isin(
    
)]
my_favorit_ids = favorite_movie['movie_id'].unique()
my_favoritgenre = favorite_movie['genre'].unique()

array([3408, 2355, 3105, 3578, 3300])

In [150]:
# 내가 좋아 하는 영화(...그렇다고 치자)
favorite_movie = ['erin brockovich (2000)', 'awakenings (1990)', 'gladiator (2000)', 'pitch black (2000)', "bug's life, a (1998)"]

# '6041'이라는 user_id가 위 영화를 7회씩 봤다고 가정
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'title': favorite_movie, 'counts':[7]*5})
my_playlist

,user_id,title,counts
0,6041,erin brockovich (2000),7
1,6041,awakenings (1990),7
2,6041,gladiator (2000),7
3,6041,pitch black (2000),7
4,6041,"bug's life, a (1998)",7


In [153]:
if not traing_data.isin({'user_id':['6041']})['user_id'].any():  # user_id에 '6041'이라는 데이터가 없다면
    traing_data = traing_data.append(my_playlist)                          # 위에 임의로 만든 my_favorite 데이터를 추가해 줍니다. 

traing_data.tail(10) # 잘 추가되었는지 확인해 봅시다.

,user_id,title,counts
836473,6040,platoon (1986),3
836474,6040,"crying game, the (1992)",5
836475,6040,welcome to the dollhouse (1995),5
836476,6040,sophie's choice (1982),4
836477,6040,e.t. the extra-terrestrial (1982),4
0,6041,erin brockovich (2000),7
1,6041,awakenings (1990),7
2,6041,gladiator (2000),7
3,6041,pitch black (2000),7
4,6041,"bug's life, a (1998)",7


In [155]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = traing_data['user_id'].unique()
movie_unique = traing_data['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)} #user_id인 v는 index가 되었고, 0부터 시작하는 index인 k는 값이 됨. 
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [156]:
len(user_unique) #고유한 유저수는 6040명

6040

In [157]:
user_to_idx[6041] #인덱스로 따지자면 마지막에 추가한 6041은 6039인덱스를 가지고 있음. index는 0부터 시작하므로

6039

In [158]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx[6041])    # 6040명의 유저 중 마지막으로 추가된 유저이니 6039가 나와야 합니다. 
print(movie_to_idx["bug's life, a (1998)"])

6039
4


In [159]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = traing_data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(traing_data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    traing_data['user_id'] = temp_user_data   # ratings['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = traing_data['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(traing_data):
    print('movie column indexing OK!!')
    traing_data['title'] = temp_movie_data
else:
    print('movie column indexing Fail!!')

traing_data

user_id column indexing OK!!
movie column indexing OK!!


,user_id,title,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5
...,...,...,...
0,6039,3,7
1,6039,18,7
2,6039,67,7
3,6039,902,7


## 4) CSR matrix를 직접 만들어 봅시다.

In [160]:
traing_data.head()

,user_id,title,counts
0,0,0,5
1,0,1,3
2,0,2,3
3,0,3,4
4,0,4,5


In [161]:
from scipy.sparse import csr_matrix

num_user = traing_data['user_id'].nunique()
num_movie = traing_data['title'].nunique()

csr_data = csr_matrix((traing_data.counts, (traing_data.user_id, traing_data.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 5) als_model = AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜 봅시다.

In [162]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [163]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T 
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [164]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 6) 내가 선호하는 5가지 영화 중 하나와 그 외의 영화 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해 보세요.

### 내가 선호하는 영화중 하나 선호도 예측 58%

In [165]:
mamagehedon, movie_item = user_to_idx[6041], movie_to_idx["bug's life, a (1998)"]
mamagehedon_vector, movie_item_vector = als_model.user_factors[mamagehedon], als_model.item_factors[movie_item]

print('슝=3')

슝=3


In [166]:
mamagehedon_vector

array([-5.5914283e-01, -7.2550988e-01, -6.8630439e-01,  8.5414565e-01,
       -3.4215873e-01,  4.1331741e-01,  5.3187168e-01,  3.1340057e-01,
        9.8439199e-01, -5.4147708e-01,  2.9542241e-01,  4.2892051e-01,
        6.0200685e-01, -6.9548452e-01, -1.0368316e+00,  4.6995699e-01,
        1.5599932e-01,  3.1712765e-01,  7.3527426e-01,  3.2376939e-01,
       -4.7723475e-01, -3.8851467e-01, -1.7949020e+00,  2.8417423e-01,
       -5.9430426e-01, -1.8048435e-01,  2.8559420e-01, -8.3076790e-02,
       -1.1161085e+00,  1.3450612e-01,  4.0702996e-01, -1.4148654e-01,
       -2.4171941e-01, -6.2986654e-01, -1.8421546e-01,  2.2584006e-01,
        1.0486276e+00,  6.1276913e-01, -7.5748885e-01, -1.8874368e-01,
        8.3024454e-01,  1.5726537e-03, -1.5418673e-01,  1.2905334e+00,
        3.2240719e-01, -5.8602417e-01,  6.2107694e-01, -1.5304908e-01,
        5.4345883e-02, -9.4028515e-01, -1.1169868e-01,  1.0426650e-02,
        1.1575360e-01, -2.2552949e-01,  1.3147351e-01,  2.1656165e-01,
      

In [167]:
movie_item_vector #bug's life, a (1998)

array([ 0.00526372, -0.00190656,  0.01639144,  0.04596758, -0.00970497,
        0.01264349,  0.0161021 ,  0.01293871, -0.0015653 ,  0.01124275,
        0.01467956,  0.01182146,  0.00946019, -0.00955878, -0.04047442,
        0.02405523,  0.0239961 , -0.0100393 , -0.00917579,  0.0082103 ,
       -0.02818336, -0.01139968, -0.02271437,  0.01386769, -0.00267211,
       -0.00608626,  0.05269356, -0.0212436 , -0.04825698, -0.02072966,
        0.00568864, -0.00326668,  0.0216603 ,  0.01204054,  0.00722583,
       -0.02549646,  0.00407256,  0.03108404,  0.0126536 ,  0.00649113,
        0.01183315, -0.00164192, -0.02000663,  0.00986787,  0.0357926 ,
       -0.0237856 ,  0.01619723,  0.0255569 ,  0.00464853, -0.01689981,
        0.0306253 ,  0.00875146, -0.01128792, -0.03742198, -0.01179373,
        0.04066668,  0.02285733, -0.04363379, -0.00810974,  0.00062144,
        0.01464024,  0.0146153 , -0.02197569,  0.00373608,  0.0089693 ,
       -0.04669205, -0.01629874, -0.02058184, -0.00710162, -0.02

In [168]:
# 유저와 선택된 영화와 내적하는 코드
np.dot(mamagehedon_vector, movie_item_vector)

0.587676

### 그 외 영화 선호도 예측 17%

In [184]:
movies[movies['title'].str.contains('Good Will')]

,movie_id,title,genre
1656,1704,Good Will Hunting (1997),Drama


In [185]:
queen = movie_to_idx['good will hunting (1997)']
queen_vector = als_model.item_factors[queen]
np.dot(mamagehedon_vector, queen_vector)

0.17176725

## 7) 내가 좋아하는 영화와 비슷한 영화를 추천받아 봅시다.

In [188]:
favorite_movie = "bug's life, a (1998)"
movie_id = movie_to_idx[favorite_movie]
similar_movies = als_model.similar_items(movie_id, N=15)
similar_movies

[(4, 1.0),
 (50, 0.69585335),
 (40, 0.6063295),
 (851, 0.44779623),
 (30, 0.41967383),
 (33, 0.4140181),
 (1706, 0.36310682),
 (10, 0.35821086),
 (841, 0.34842688),
 (34, 0.34632212),
 (1635, 0.34540367),
 (793, 0.3399471),
 (126, 0.33729872),
 (212, 0.32503545),
 (3496, 0.3084447)]

In [189]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movies]

["bug's life, a (1998)",
 'toy story 2 (1999)',
 'toy story (1995)',
 'iron giant, the (1999)',
 'antz (1998)',
 'aladdin (1992)',
 'stuart little (1999)',
 'beauty and the beast (1991)',
 'prince of egypt, the (1998)',
 'mulan (1998)',
 'dinosaur (2000)',
 'american pie (1999)',
 'shakespeare in love (1998)',
 'bowfinger (1999)',
 'voyage to the beginning of the world (1997)']

In [190]:
def get_similar_artist(artist_name: str):
    movie_id = movie_to_idx[artist_name]
    similar_movies = als_model.similar_items(movie_id)
    similar_movies = [idx_to_movie[i[0]] for i in similar_movies]
    return similar_movies

print("슝=3")

슝=3


In [191]:
get_similar_artist('good will hunting (1997)')

['good will hunting (1997)',
 'apollo 13 (1995)',
 'shawshank redemption, the (1994)',
 'truman show, the (1998)',
 'dead man walking (1995)',
 'pulp fiction (1994)',
 'jerry maguire (1996)',
 "what's eating gilbert grape (1993)",
 'searching for bobby fischer (1993)',
 'october sky (1999)']

## 8) 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.

In [194]:
user = user_to_idx[6041]
# recommend에서는 user*item CSR Matrix를 받습니다.
movies_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movies_recommended

[(50, 0.50939),
 (346, 0.48361987),
 (463, 0.44686338),
 (482, 0.43841377),
 (782, 0.35098457),
 (637, 0.3458599),
 (40, 0.32239076),
 (1537, 0.3189805),
 (474, 0.31362736),
 (444, 0.30578786),
 (372, 0.28795758),
 (327, 0.28312576),
 (293, 0.2619971),
 (460, 0.2586131),
 (746, 0.25335553),
 (300, 0.24753246),
 (468, 0.23721682),
 (479, 0.2354853),
 (164, 0.22964923),
 (391, 0.22518326)]

In [195]:
[idx_to_movie[i[0]] for i in movies_recommended]

['toy story 2 (1999)',
 'patriot, the (2000)',
 'frequency (2000)',
 'mission: impossible 2 (2000)',
 'mission to mars (2000)',
 'abyss, the (1989)',
 'toy story (1995)',
 'u-571 (2000)',
 'chicken run (2000)',
 'high fidelity (2000)',
 'x-men (2000)',
 'gone in 60 seconds (2000)',
 'final destination (2000)',
 'boiler room (2000)',
 'perfect storm, the (2000)',
 'shanghai noon (2000)',
 'magnolia (1999)',
 'contact (1997)',
 '28 days (2000)',
 'cider house rules, the (1999)']

### 모델이 추천한 영화가 실제로 선호하는지 계산하여 모델의 객관적 지표 도출

In [199]:
apollo = movie_to_idx['toy story 2 (1999)']
explain = als_model.explain(user, csr_data, itemid=apollo)

In [200]:
[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[("bug's life, a (1998)", 0.42498677034231785),
 ('pitch black (2000)', 0.057810745316481324),
 ('erin brockovich (2000)', 0.04073484118092531),
 ('awakenings (1990)', -0.0009592355378706206),
 ('gladiator (2000)', -0.020106779288773816)]

## 느낀점
추천시스템을 배울 수 있는 좋은 시간이였다. 
예전에 검색엔진 모듈을 심는 작업을 한 적이 있었는데, 그때와 비슷한 개념들이 잠깐 나와서 좀 더 흥미있게 봤던거 같다. 
프로젝트 가이드에 rating 컬럼의 이름을 count로 바꾸라는 지시가 있었는데, count로 바꾸면 csr_matrix에서 에러가 발생한다. 
아마도 cst_matrix()에서 count를 함수로 인식해버린 듯 하다. 
작가의 의도인거 같다. 오픈소스에 기여할 수 있는 기회(?)를 만들어준게 아닐까?
모델을 만들고 예측을 돌려봤지만, 다 오래된 영화들이라 내 선호도에 맞는지 정확하게 알 수가 없다. 
아무리 데이터로 지표를 제공해도 사람의 속은 모르는 것이니까...
last.fm 사이트도 알게 되고 흥미로운 시간이였다. 